In [1]:
import torch
import torchvision
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets 
from torchvision import transforms
from torchvision.utils import save_image
from torch.autograd import Variable


import pandas as pd 
import numpy as np
import os 
import glob
np.random.seed(1337)  # for reproducibility
from sklearn import preprocessing
from sklearn.model_selection import train_test_split
import torch.optim as optim
import torch
import torch.nn.functional as nn
import torch.autograd as autograd
import torch.optim as optim
import numpy as np
#import matplotlib.pyplot as plt
#import matplotlib.gridspec as gridspec
import os
from torch.autograd import Variable
from sklearn.utils import shuffle
import random
from keras.datasets import mnist
from keras.callbacks import EarlyStopping
from keras.callbacks import ModelCheckpoint
from keras.models import Sequential
from keras.layers import Dense, Dropout, Activation, Flatten
from keras.layers import Convolution2D, MaxPooling2D
from keras.utils import np_utils
from keras import backend as K
from keras import optimizers

Using TensorFlow backend.


In [2]:
X_train = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_10sec_ivectors_X_train_29Dec.csv')
#train_afds

In [3]:
#train_lre = train_lre.iloc[1000:2000]

In [4]:
#train_lre.head()

In [5]:
y_train = pd.read_csv('/home/satishk/GAN_lre/gan_csv/GAN_30sec_ivectors_Y_train_29Dec.csv')


In [6]:
X_train=X_train.values
y_train=y_train.values

In [7]:
X_train = X_train.astype('float32')
#X_test = X_test.astype('float32')
y_train = y_train.astype('float32')

#X_train /= 255
#X_test /= 255
#X_val /= 255

print('X_train shape:', X_train.shape)
print(X_train.shape[0], 'train 10sec')
#print(X_test.shape[0], 'test samples')
print(y_train.shape[0], ' train 30sec')


X_train shape: (114301, 500)
114301 train 10sec
114301  train 30sec


In [8]:
#Shuffle the Dataset
X_train,  y_train = shuffle(X_train, y_train, random_state=0)

In [10]:
import torch.nn as nn

In [44]:
#Implementing Generator as Encoder-Decoder pair inspired from DAGAN base paper

#Encoder should be able to take a batch of input(of dim 500-ivector) and be able to produce its representation r

class Encoder(nn.Module):
    def __init__(self):
        super(Encoder, self).__init__()
        self.fc1 = nn.Linear(500, 512)
        self.fc2 = nn.Linear(512, 256)
        self.fc3 = nn.Linear(256, 128)
    def forward(self, x):
        return self.fc3(F.relu(self.fc2(F.relu(self.fc1(x)))))
    
#We will use Decoder as Generator    
#Decoder should be able to take input of dim 128(r) and 100(z),concatenate r and z and 
#produce an output od dim 500-ivector    

class Decoder(nn.Module):
    def __init__(self):
        super(Decoder, self).__init__()
        self.fc1 = nn.Linear(128,256 )
        self.fc2 = nn.Linear(256, 512)
        self.fc3 = nn.Linear(512,500)
    def forward(self, inputs):
        inputs = inputs #torch.cat([z, x], 1)
        return (self.fc3(F.relu(self.fc2(F.relu(self.fc1(inputs))))))

#input to Generator alias Decoder is inputs = torch.cat([z, r], 1)
#Where r is the output of encoder given x i.e., representation of x encoded by encoder


In [45]:
#As we defined the Encoder and Decoder we now Implement the Generator 

class Generator(nn.Module):
                                
    def __init__(self):
        super(Generator, self).__init__()
        self.fc1 = Encoder()
        self.fc2 = Decoder()

    def forward(self, x):
        inputs = x   #torch.cat([z, x], 1)
        return self.fc2(self.fc1(inputs))

In [46]:
G = Generator()
#D = Discriminator()

In [47]:
G.modules

<bound method Module.modules of Generator(
  (fc1): Encoder(
    (fc1): Linear(in_features=500, out_features=512)
    (fc2): Linear(in_features=512, out_features=256)
    (fc3): Linear(in_features=256, out_features=128)
  )
  (fc2): Decoder(
    (fc1): Linear(in_features=128, out_features=256)
    (fc2): Linear(in_features=256, out_features=512)
    (fc3): Linear(in_features=512, out_features=500)
  )
)>

In [48]:
learning_rate = 0.0002
betas = (0.5, 0.999)

In [49]:
# Loss function
criterion = torch.nn.MSELoss()

# Optimizers
G_solver = torch.optim.Adam(G.parameters(), lr=learning_rate, betas=betas)
#D_solver = torch.optim.Adam(D.parameters(), lr=learning_rate/2, betas=betas)

In [50]:
len(X_train)

114301

In [51]:
len(y_train)

114301

In [52]:

mb_size = 256
batch_size = mb_size
# Start training
for epoch in range(100):
    
    

    print('epoch:',epoch)
    #for i in range(XX_train):
    # Build mini-batch dataset
    #batch_size = images.size(0)
    #images = to_var(images.view(batch_size, -1))

    it=0
    while it+batch_size < len(X_train) :
        

        start= it
        end= it + batch_size


        #z = Variable(torch.randn(mb_size, Z_dim))
        X = X_train[start:end]

        c = y_train[start:end]
        X = Variable(torch.from_numpy(X))
        c = Variable(torch.from_numpy(c.astype('float32')))

        # Dicriminator forward-loss-backward-update
        G_sample = G(X)
        #D_real = D(X, c)
        #D_fake = D(G_sample, c)

        #D_loss_real = nn.binary_cross_entropy(D_real, ones_label)
        #D_loss_fake = nn.binary_cross_entropy(D_fake, zeros_label)
        #D_loss = D_loss_real + D_loss_fake

        #D_loss.backward()
        #D_solver.step()

        # Housekeeping - reset gradient
        #D.zero_grad()

        # Generator forward-loss-backward-update
        #z = Variable(torch.randn(mb_size, Z_dim))
        #G_sample = G(z, c)
        #D_fake = D(G_sample, c)
        G_loss = criterion(G_sample, c)
        #G_loss = nn.binary_cross_entropy(D_fake, ones_label)

        G_loss.backward()
        G_solver.step()
        
        G_solver.zero_grad()
        # Housekeeping - reset gradient
        #D.zero_grad()

        #Print and plot every now and then
        #if it % 2 == 0:

        #print('Iter-{}; G_loss: {}'.format(it, G_loss.data.numpy()))



        it+= batch_size
    print('G_loss: {}'.format(G_loss.data.numpy()))
           

epoch: 0
G_loss: [ 1.44936967]
epoch: 1
G_loss: [ 1.41561997]
epoch: 2
G_loss: [ 1.39547551]
epoch: 3
G_loss: [ 1.38132727]
epoch: 4
G_loss: [ 1.369802]
epoch: 5
G_loss: [ 1.35914612]
epoch: 6
G_loss: [ 1.34941649]
epoch: 7
G_loss: [ 1.33964336]
epoch: 8
G_loss: [ 1.33025551]
epoch: 9
G_loss: [ 1.32210565]
epoch: 10
G_loss: [ 1.31454349]
epoch: 11
G_loss: [ 1.30790794]
epoch: 12
G_loss: [ 1.30206597]
epoch: 13
G_loss: [ 1.29730821]
epoch: 14
G_loss: [ 1.2934252]
epoch: 15
G_loss: [ 1.29019582]
epoch: 16
G_loss: [ 1.28744411]
epoch: 17
G_loss: [ 1.2845943]
epoch: 18
G_loss: [ 1.28209448]
epoch: 19
G_loss: [ 1.27992952]
epoch: 20
G_loss: [ 1.27805114]
epoch: 21
G_loss: [ 1.27637017]
epoch: 22
G_loss: [ 1.27482438]
epoch: 23
G_loss: [ 1.27343392]
epoch: 24
G_loss: [ 1.27208364]
epoch: 25
G_loss: [ 1.27084422]
epoch: 26
G_loss: [ 1.26957476]
epoch: 27
G_loss: [ 1.26802647]
epoch: 28
G_loss: [ 1.26651645]
epoch: 29
G_loss: [ 1.26515412]
epoch: 30
G_loss: [ 1.26378334]
epoch: 31
G_loss: [ 1.

In [ ]:
df_SF=pd.DataFrame()
samples_per_class = 1000
#c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
#c[:, np.random.randint(0, 10)] = 1.
for i in range(14):
    #print(i)
    c = np.zeros(shape=[samples_per_class, y_dim], dtype='float32')
    c[:, i] = 1.
    c_df=pd.DataFrame(c)
    df_SF = df_SF.append(c_df,ignore_index = True)
    
    

In [ ]:
c_df.shape

In [ ]:
df_SF.shape[0]

In [ ]:
c_gen = df_SF.values

In [ ]:
z_gen = Variable(torch.randn(df_SF.shape[0], Z_dim))

In [ ]:
z_gen.data.shape

In [ ]:
c_gen = Variable(torch.from_numpy(c_gen))

In [ ]:
samples = G(z_gen, c_gen)

In [ ]:
samples.data.shape

In [ ]:
#Taking the generated iVectors we will try to check the acc by MLP

In [ ]:
train_X1 = samples.data.numpy()

In [ ]:
type(train_X1)

In [ ]:
train_y1 = c_gen.data.numpy()

In [ ]:
type(train_y1)

In [ ]:
train_y1

In [ ]:
X_train = pd.DataFrame(X_train)
Y_train = pd.DataFrame(Y_train)


In [ ]:
train_X1 = pd.DataFrame(train_X1)
train_y1 = pd.DataFrame(train_y1)

In [ ]:
#train_y

In [ ]:
train_X = X_train.append(train_X1, ignore_index=True)
train_y = Y_train.append(train_y1, ignore_index=True)

In [ ]:
#train_X,  train_y = shuffle(train_X, train_y, random_state=0)

In [ ]:
train_X = train_X.values
train_y = train_y.values

In [ ]:
train_y

In [ ]:
model = Sequential()
model.add(Dense(512, input_dim=500))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(512))
model.add(Activation('relu'))
model.add(Dropout(0.5))

model.add(Dense(nb_classes))
model.add(Activation('softmax'))

In [ ]:
model.summary()

In [ ]:
adam=optimizers.Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)

model.compile(loss='categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

In [ ]:
best_weights_filepath = '/home/satishk/saved_weights/best_weights_2l_MLP_11.hdf5'
saveBestModel = ModelCheckpoint(best_weights_filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='auto')


In [ ]:
batch_size = 256
nb_epoch=30

In [ ]:
#Checking Accuracy with training+augmented data train_X and train_y are 'train + augmented' data
history = model.fit(train_X, train_y, batch_size=batch_size, epochs=nb_epoch,verbose=2, 
                    validation_data=(X_test , y_test),callbacks=[saveBestModel])

In [ ]:
#Frame label accuracy
score = model.evaluate(X_test, y_test, verbose=0)
print('ERROR after Data Augmentation %:', 1-score[1])